In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [ ]:
(c100_x_train, c100_y_train), (c100_x_test, c100_y_test) = tf.keras.datasets.cifar100.load_data()
(c10_x_train, c10_y_train), (c10_x_test, c10_y_test) = tf.keras.datasets.cifar10.load_data()

x_train = np.concatenate((c10_x_train, c100_x_train), axis=0)
x_test = np.concatenate((c10_x_test, c100_x_test), axis=0)
y_train = np.concatenate((c10_y_train, np.ones(c100_y_train.shape, dtype=np.int) * 10), axis=0)
y_test = np.concatenate((c10_y_test, np.ones(c100_y_test.shape, dtype=np.int) * 10), axis=0)

x_train, x_test = x_train.astype('float32') / 255.0, x_test.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train, 11)
y_test = tf.keras.utils.to_categorical(y_test, 11)

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=False)

datagen.fit(x_train)


In [ ]:
print(x_train.shape)
print(c100_x_train.shape)
print(c10_x_train.shape)

(100000, 32, 32, 3)
(50000, 32, 32, 3)
(50000, 32, 32, 3)


In [ ]:
def cnn_model():
    
    model = Sequential()
    
    # First Conv layer
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4), input_shape=(32,32,3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    # Second Conv layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    # Third, fourth, fifth convolution layer
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(1e-4)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    # Fully Connected layers
    model.add(Flatten())
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(10, activation='softmax'))
    
    model.summary()
    
    return model

model = cnn_model()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 256)       295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 256)         0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 512)        

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0003, decay=1e-6),
             metrics=['accuracy'])

history = model.fit_generator(datagen.flow(x_train, y_train, batch_size = 64),
                    steps_per_epoch = len(x_train) // 64, 
                    epochs = 125, 
                    validation_data= (x_test, y_test),
                    verbose=1)

print(model.evaluate(X_test, y_test))

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/125
781/781 [==============================] - 2271s 3s/step - loss: 2.0991 - accuracy: 0.2188 - val_loss: 1.7234 - val_accuracy: 0.3734
Epoch 2/125
654/781 [========================>.....] - ETA: 5:51 - loss: 1.6022 - accuracy: 0.4085